In [1]:
!python3 -m pip install xmltodict numpy pandas matplotlib mne scikit-learn scipy joblib autoreject tqdm PyWavelets spectrum xgboost seaborn mock plotly

     |████████████████████████████████| 6.4MB 2.8MB/s 
     |████████████████████████████████| 235kB 56.7MB/s 
  Created wheel for spectrum: filename=spectrum-0.7.6-cp36-cp36m-linux_x86_64.whl size=234245 sha256=30b81c3e548f1f39f715bee2fda648f2e923115e63854e2edc4ed8d91ba562d9
  Stored in directory: /root/.cache/pip/wheels/7b/a1/1f/16e3bd0418dc16201a4f2e696ab00de3e3c95549cba7df5d13
Successfully built spectrum


In [0]:
#%% #* Import statements
import os
import numpy as np
import pandas as pd
import scipy.stats as sp
import matplotlib.pyplot as plt
import mne
from tqdm import tqdm as tqdm
from spectrum import arburg
from autoreject import AutoReject
import mock
import sklearn.preprocessing as skpr

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
%matplotlib inline

In [0]:
DATA_DIR = '/content/drive/My Drive/mahnob/Sessions'

def getDataFiles(rootFolder):
  sessionFolders, sessions = list(sorted(os.listdir(rootFolder))), []
  for sessionFolder in sessionFolders:
    _path = os.path.join(DATA_DIR, sessionFolder)
    if os.path.isdir(_path):
      node = {'folder': sessionFolder}
      for subfile in os.listdir(_path):
        if subfile.endswith('.bdf'):
          node['bdf'] = subfile
        elif subfile.endswith('.xml'):
          node['xml'] = subfile
      if 'bdf' in node and 'xml' in node and 'S_Trial' in node['bdf']:
        sessions.append(node)
  return sessions

In [0]:
def concat_features(save_as):
  inp = pd.DataFrame()
  sessions = getDataFiles(DATA_DIR)
  for session in tqdm(sessions):
    each = pd.read_pickle(
        os.path.join(DATA_DIR, session['folder'], 'features.pkl'))
    inp = pd.concat([inp, each])
  inp.to_pickle(os.path.join(DATA_DIR, save_as))

In [0]:
## Run this for concating the features from each Session
# concat_features('data_935.pkl')

In [0]:
inp = pd.DataFrame(pd.read_pickle(os.path.join(DATA_DIR, 'data_45.pkl')))

In [8]:
inp

,coeffVar,AR1_mean,AR1_min,AR1_max,AR2_mean,AR2_min,AR2_max,AR3_mean,AR3_min,AR3_max,hjorthActivity,hjorthMobility,hjorthComplexity,PMax1,PMax2,PMax3,PMax4,PRatio1,PRatio2,PRatio3,PRatio4,cA_mean_mean,cA_mean_min,cA_mean_max,cA_std_mean,cA_std_min,cA_std_max,cA_Energy_mean,cA_Energy_min,cA_Energy_max,cD_mean_mean,cD_mean_min,cD_mean_max,cD_std_mean,cD_std_min,cD_std_max,cD_Energy_mean,cD_Energy_min,cD_Energy_max,Entropy_D_mean,Entropy_D_min,Entropy_D_max,Entropy_A_mean,Entropy_A_min,Entropy_A_max,valence,arousal,control,prediction
0,-0.000817,-1.516150,1.037651,-0.521501,-1.779246,0.646605,-0.907205,-1.190684,1.565949,-0.309605,217.933527,0.199901,6.520618,19.300715,1.829833,1.083941,0.481631,10.547800,17.806055,3.799241,19.494184,-12821.133495,-31793.603170,14739.388232,19.282207,5.663456,39.774956,1.130111e+11,2.554023e+09,3.901820e+11,-0.003142,-0.024549,0.023169,1.392032,0.648588,2.999578,836.242621,162.412349,3473.126215,1407.562759,8.348266,8937.473975,2.246619e+12,4.011125e+10,8.090051e+12,3,4,1,5
0,-0.000605,-1.494602,1.062716,-0.568113,-1.757999,0.652676,-0.916241,-1.172010,1.564404,-0.362611,108.724995,0.263973,4.928193,10.452956,1.494933,0.906143,0.420375,6.992258,11.535653,3.556191,13.185428,-12846.660013,-31869.607386,14702.752900,13.617285,5.725770,27.284464,1.133107e+11,2.531537e+09,3.920495e+11,0.000936,-0.039294,0.029792,1.356697,0.595335,3.025144,798.689180,136.833015,3532.597607,1278.835095,15.983772,9235.384020,2.252982e+12,3.973567e+10,8.130643e+12,3,4,1,5
0,-0.000651,-1.536925,1.137604,-0.600679,-1.738083,0.752656,-0.913852,-1.281181,1.586035,-0.397356,135.138115,0.265776,5.155265,9.056223,1.683857,1.152212,0.457497,5.378262,7.859860,3.680585,9.321273,-12841.613508,-31822.091004,14705.079831,14.656776,5.267933,38.978936,1.132730e+11,2.522294e+09,3.908814e+11,-0.004128,-0.056521,0.029740,1.409607,0.647855,3.021783,855.834945,162.058955,3524.632847,1420.347023,14.558757,9207.943638,2.252110e+12,3.958136e+10,8.105252e+12,3,4,1,5
0,-0.000637,-1.488197,1.053504,-0.565307,-1.732677,0.698738,-0.917893,-1.253546,1.558658,-0.354796,128.923273,0.273854,5.178132,7.616664,1.865179,0.920780,0.451719,4.083610,8.271966,4.129068,10.297616,-12826.782277,-31838.200919,14708.344069,14.079137,4.811251,34.108953,1.130886e+11,2.495521e+09,3.912774e+11,-0.001913,-0.023708,0.025562,1.368587,0.642069,3.128015,826.526747,159.133673,3776.963196,1375.468270,7.732333,10195.642259,2.248334e+12,3.913459e+10,8.113860e+12,3,4,1,5
0,-0.000976,-1.502020,1.062290,-0.560270,-1.709642,0.670793,-0.915898,-1.215146,1.563960,-0.363738,189.579325,0.212530,6.833778,25.551277,2.408184,0.904731,0.455597,10.610183,28.241847,5.285780,30.903615,-12848.585745,-31911.649445,14718.878426,17.867863,5.346426,39.169386,1.134440e+11,2.479611e+09,3.930845e+11,-0.004857,-0.041841,0.019457,1.330958,0.634201,3.073831,775.461205,155.282753,3647.243411,1258.018833,17.104093,9626.034285,2.255876e+12,3.886926e+10,8.153145e+12,3,4,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.190341,-1.322248,0.849287,-0.525464,-1.670404,0.320470,-0.998301,-0.883189,1.668705,-0.275140,180.939797,0.442962,3.602322,38.417918,9.096773,1.713247,1.028182,4.223247,22.424035,8.847431,27.733701,-6292.201511,-30075.126042,16282.194200,16.154479,0.914189,44.861241,5.010855e+10,2.013165e+04,3.491424e+11,0.029894,-0.060971,0.248113,3.098928,0.344344,16.379951,6770.210161,45.951470,103565.106927,31628.621060,22.123645,611801.959589,9.843278e+11,9.526675e+04,7.200333e+12,7,4,7,4
0,-0.014746,-1.306751,0.826208,-0.518576,-1.671154,0.331910,-0.997957,-0.857758,1.669110,-0.211702,191.622311,0.475771,3.184522,34.730412,14.045661,2.473135,0.973213,2.472679,14.043070,14.432265,19.722364,-6298.204471,-30064.509036,16251.428913,16.333949,1.113821,40.938902,5.010424e+10,4.558315e+04,3.488961e+11,-0.005562,-0.331680,0.104796,3.272265,0.347553,16.3094

In [0]:
emo_dims = ['valence', 'arousal', 'control', 'prediction']

inp_emo_dims = inp[emo_dims]

inp = inp.drop(emo_dims, axis=1)

# inp = (inp - inp.mean()) / inp.std()

# inp_emo_dims = inp_emo_dims.groupby('subjectid').transform(lambda x: (x - x.mean()) / x.std())

inp[emo_dims] = inp_emo_dims

In [10]:
inp

,coeffVar,AR1_mean,AR1_min,AR1_max,AR2_mean,AR2_min,AR2_max,AR3_mean,AR3_min,AR3_max,hjorthActivity,hjorthMobility,hjorthComplexity,PMax1,PMax2,PMax3,PMax4,PRatio1,PRatio2,PRatio3,PRatio4,cA_mean_mean,cA_mean_min,cA_mean_max,cA_std_mean,cA_std_min,cA_std_max,cA_Energy_mean,cA_Energy_min,cA_Energy_max,cD_mean_mean,cD_mean_min,cD_mean_max,cD_std_mean,cD_std_min,cD_std_max,cD_Energy_mean,cD_Energy_min,cD_Energy_max,Entropy_D_mean,Entropy_D_min,Entropy_D_max,Entropy_A_mean,Entropy_A_min,Entropy_A_max,valence,arousal,control,prediction
0,-0.000817,-1.516150,1.037651,-0.521501,-1.779246,0.646605,-0.907205,-1.190684,1.565949,-0.309605,217.933527,0.199901,6.520618,19.300715,1.829833,1.083941,0.481631,10.547800,17.806055,3.799241,19.494184,-12821.133495,-31793.603170,14739.388232,19.282207,5.663456,39.774956,1.130111e+11,2.554023e+09,3.901820e+11,-0.003142,-0.024549,0.023169,1.392032,0.648588,2.999578,836.242621,162.412349,3473.126215,1407.562759,8.348266,8937.473975,2.246619e+12,4.011125e+10,8.090051e+12,3,4,1,5
0,-0.000605,-1.494602,1.062716,-0.568113,-1.757999,0.652676,-0.916241,-1.172010,1.564404,-0.362611,108.724995,0.263973,4.928193,10.452956,1.494933,0.906143,0.420375,6.992258,11.535653,3.556191,13.185428,-12846.660013,-31869.607386,14702.752900,13.617285,5.725770,27.284464,1.133107e+11,2.531537e+09,3.920495e+11,0.000936,-0.039294,0.029792,1.356697,0.595335,3.025144,798.689180,136.833015,3532.597607,1278.835095,15.983772,9235.384020,2.252982e+12,3.973567e+10,8.130643e+12,3,4,1,5
0,-0.000651,-1.536925,1.137604,-0.600679,-1.738083,0.752656,-0.913852,-1.281181,1.586035,-0.397356,135.138115,0.265776,5.155265,9.056223,1.683857,1.152212,0.457497,5.378262,7.859860,3.680585,9.321273,-12841.613508,-31822.091004,14705.079831,14.656776,5.267933,38.978936,1.132730e+11,2.522294e+09,3.908814e+11,-0.004128,-0.056521,0.029740,1.409607,0.647855,3.021783,855.834945,162.058955,3524.632847,1420.347023,14.558757,9207.943638,2.252110e+12,3.958136e+10,8.105252e+12,3,4,1,5
0,-0.000637,-1.488197,1.053504,-0.565307,-1.732677,0.698738,-0.917893,-1.253546,1.558658,-0.354796,128.923273,0.273854,5.178132,7.616664,1.865179,0.920780,0.451719,4.083610,8.271966,4.129068,10.297616,-12826.782277,-31838.200919,14708.344069,14.079137,4.811251,34.108953,1.130886e+11,2.495521e+09,3.912774e+11,-0.001913,-0.023708,0.025562,1.368587,0.642069,3.128015,826.526747,159.133673,3776.963196,1375.468270,7.732333,10195.642259,2.248334e+12,3.913459e+10,8.113860e+12,3,4,1,5
0,-0.000976,-1.502020,1.062290,-0.560270,-1.709642,0.670793,-0.915898,-1.215146,1.563960,-0.363738,189.579325,0.212530,6.833778,25.551277,2.408184,0.904731,0.455597,10.610183,28.241847,5.285780,30.903615,-12848.585745,-31911.649445,14718.878426,17.867863,5.346426,39.169386,1.134440e+11,2.479611e+09,3.930845e+11,-0.004857,-0.041841,0.019457,1.330958,0.634201,3.073831,775.461205,155.282753,3647.243411,1258.018833,17.104093,9626.034285,2.255876e+12,3.886926e+10,8.153145e+12,3,4,1,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,0.190341,-1.322248,0.849287,-0.525464,-1.670404,0.320470,-0.998301,-0.883189,1.668705,-0.275140,180.939797,0.442962,3.602322,38.417918,9.096773,1.713247,1.028182,4.223247,22.424035,8.847431,27.733701,-6292.201511,-30075.126042,16282.194200,16.154479,0.914189,44.861241,5.010855e+10,2.013165e+04,3.491424e+11,0.029894,-0.060971,0.248113,3.098928,0.344344,16.379951,6770.210161,45.951470,103565.106927,31628.621060,22.123645,611801.959589,9.843278e+11,9.526675e+04,7.200333e+12,7,4,7,4
0,-0.014746,-1.306751,0.826208,-0.518576,-1.671154,0.331910,-0.997957,-0.857758,1.669110,-0.211702,191.622311,0.475771,3.184522,34.730412,14.045661,2.473135,0.973213,2.472679,14.043070,14.432265,19.722364,-6298.204471,-30064.509036,16251.428913,16.333949,1.113821,40.938902,5.010424e+10,4.558315e+04,3.488961e+11,-0.005562,-0.331680,0.104796,3.272265,0.347553,16.3094

In [12]:
px.scatter_matrix(inp,
    dimensions=['valence', 'arousal', 'control', 'prediction'])

In [13]:
px.imshow(inp[['valence', 'arousal', 'control', 'prediction']].corr())

In [0]:
emo_dims = ['valence', 'arousal', 'control', 'prediction']
Y = inp[emo_dims]
X = inp.drop(columns=emo_dims)

In [0]:
Y2 = pd.DataFrame()
for emo_dim in emo_dims:
  Y2[emo_dim] = Y.apply(lambda row: 'low' if row[emo_dim] <= 4.5 else 'high', axis=1)

In [0]:
def get_model(X, Y):
  model_fi = []
  for emotion in emo_dims:
    print('\n\t+++ Model for {}'.format(emotion))
    y = Y[emotion]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    model = XGBClassifier()
    model.fit(X_train, y_train)
    predict_train = model.predict(X_train)
    accuracy_train = accuracy_score(y_train, predict_train)
    print('accuracy_score on train dataset : ', accuracy_train)
    predict_test = model.predict(X_test)
    accuracy_test = accuracy_score(y_test, predict_test)
    print('accuracy_score on test dataset : ', accuracy_test)
    model_fi.append(pd.Series(model.feature_importances_, index=X.columns, name=emotion))
  feat_importances = pd.concat(model_fi, axis=1)
  return model, feat_importances

In [0]:
def plot_feat_imp(feat_importances, show_fig=True):
  fig = make_subplots(rows=2, cols=2)
  for i in range(2):
    for j in range(2):
      ser = feat_importances[emo_dims[i * 2 + j]].sort_values(ascending=False).nlargest(10)
      fig.add_trace(go.Bar(
          x=ser.keys(),
          y=ser,
          name=emo_dims[i * 2 + j]
      ), row = i + 1, col = j + 1)
      # fig.update_xaxes(title_text="importance", row=i + 1, col=j + 1)
      # fig.update_yaxes(title_text="feature", row=i + 1, col=j + 1)
  fig.update_layout(title_text="Feature Importance", height=1000)
  if show_fig:
    fig.show()
  else:
    return fig

In [20]:
model, feat_importances = get_model(X, Y2)


	+++ Model for valence
accuracy_score on train dataset :  0.8163674321503132
accuracy_score on test dataset :  0.7818971275885104

	+++ Model for arousal
accuracy_score on train dataset :  0.8019206680584551
accuracy_score on test dataset :  0.760187040748163

	+++ Model for control
accuracy_score on train dataset :  0.8121085594989561
accuracy_score on test dataset :  0.779559118236473

	+++ Model for prediction
accuracy_score on train dataset :  0.8208768267223382
accuracy_score on test dataset :  0.8102872411489646


In [22]:
plot_feat_imp(feat_importances)

In [0]:
X2

,coeff_var,kurtosis,skew,d1_mean,d1_max,d2_mean,d2_max,ar1,ar2,ar3,hjworth_activity,hjworth_mobility,hjworth_complexity,PMax1,PMax2,PMax3,PMax4,PRatio1,PRatio2,PRatio3,PRatio4,cA_mean,cA_std,cA_energy,cA_entropy,cD_mean,cD_std,cD_energy,cD_entropy,ar1_min,ar1_max,ar2_min,ar2_max,ar3_min,ar3_max,cA_mean_min,cA_mean_max,cA_std_min,cA_std_max,cA_energy_min,cA_energy_max,cA_entropy_min,cA_entropy_max,cD_mean_min,cD_mean_max,cD_std_min,cD_std_max,cD_energy_min,cD_energy_max,cD_entropy_min,cD_entropy_max
0,-0.148941,1.385376,0.153950,-0.004437,8.368149,-0.003784,9.529279,-1.514540,1.036467,-0.519901,217.933527,0.235994,4.503993,154.405719,10.562634,0.317756,0.173466,28.002255,626.706388,187.000315,680.765700,99.593174,19.282207,7.995540e+06,8.417833e+07,-0.003142,1.392032,836.242621,1405.269353,-1.778835,-1.190270,0.646496,1.565566,-0.906585,-0.309159,-111.164267,302.053939,5.663456,39.774956,28694.196881,3.532456e+07,151630.502936,4.037710e+08,-0.024549,0.023169,0.648588,2.999578,162.412349,3473.126215,-28.346230,8937.473975
1,-1.192374,-0.343304,0.334797,-0.013156,6.696277,0.001823,8.093735,-1.493038,1.061751,-0.566486,108.724995,0.327882,3.094050,83.623646,11.025561,0.285827,0.144937,15.374779,433.370546,201.621695,500.008585,74.066656,13.617285,6.165888e+06,6.350578e+07,0.000936,1.356697,798.689180,1274.774878,-1.757927,-1.171861,0.652634,1.561962,-0.910928,-0.362547,-137.744126,276.828931,5.725770,27.284464,37981.168275,2.970339e+07,200441.934597,3.344365e+08,-0.039294,0.029792,0.595335,3.025144,136.833015,3532.597607,-48.979698,9235.384020
2,0.014314,-0.494756,0.142554,0.016472,7.824204,0.005648,8.325180,-1.535013,1.136431,-0.598854,135.138115,0.315404,3.377651,72.449782,11.728601,0.325193,0.130765,13.645512,288.650186,181.835661,342.746829,79.113161,14.656776,6.001434e+06,6.151886e+07,-0.004128,1.409607,855.834945,1417.995086,-1.738007,-1.281036,0.748732,1.584505,-0.911472,-0.397316,-136.709028,240.174545,5.267933,38.978936,38881.710060,2.233597e+07,200174.552839,2.450747e+08,-0.056521,0.029740,0.647855,3.021783,162.058955,3524.632847,-37.630996,9207.943638
3,0.148039,-0.409854,-0.250486,-0.002587,7.254869,-0.002355,8.533187,-1.487350,1.053011,-0.564446,128.923273,0.340632,3.172374,60.933308,10.944112,0.306309,0.131431,11.241041,260.040052,210.621152,308.428107,93.944392,14.079137,5.939305e+06,6.055543e+07,-0.001913,1.368587,826.526747,1372.799625,-1.732608,-1.253525,0.697976,1.558333,-0.917517,-0.354647,-82.284666,236.298376,4.811251,34.108953,51875.185088,2.167265e+07,278521.325929,2.372422e+08,-0.023708,0.025562,0.642069,3.128015,159.133673,3776.963196,-27.382854,10195.642259
4,0.140779,-0.469670,-0.026053,-0.042189,7.947251,-0.001305,8.332992,-1.501220,1.061810,-0.559516,189.579325,0.268223,4.071339,204.723646,9.722782,0.331290,0.127602,35.912553,640.031108,228.259682,676.665618,72.140924,17.867863,4.242848e+06,4.216789e+07,-0.004857,1.330958,775.461205,1253.169827,-1.709482,-1.215033,0.669656,1.561967,-0.913903,-0.363606,-78.000841,243.762706,5.346426,39.169386,140536.228367,2.300959e+07,892889.710786,2.531496e+08,-0.041841,0.019457,0.634201,3.073831,155.282753,3647.243411,-35.240408,9626.034285
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27,1.366056,1.096667,0.416314,0.010789,17.051726,0.003682,19.338831,-1.344550,0.868985,-0.512350,356.811154,1.234323,1.288894,348.980970,116.684078,1.720213,0.373109,33.043537,237.529914,1175.566113,359.745926,31.861974,19.777957,1.898644e+06,1.744942e+07,-0.002033,3.102649,6809.913428,32009.550638,-1.674550,-0.948480,0.377742,1.666478,-0.997192,-0.242830,-101.319336,149.053325,1.024750,89.139289,28007.189879,8.636579e+06,141035.395644,8.662275e+07,-0.079634,0.075379,0.351329,16.628778,47.644739,106735.547296,-62.542062,633075.795339
28,0.257526,1.930401,0.656686,-0.009539,15.018527,-0.000904,19.556604,-1.317751,0.848833,-0.5

In [0]:
list(sorted(X2.columns))

['PMax1',
 'PMax2',
 'PMax3',
 'PMax4',
 'PRatio1',
 'PRatio2',
 'PRatio3',
 'PRatio4',
 'ar1',
 'ar1_max',
 'ar1_min',
 'ar2',
 'ar2_max',
 'ar2_min',
 'ar3',
 'ar3_max',
 'ar3_min',
 'cA_energy',
 'cA_energy_max',
 'cA_energy_min',
 'cA_entropy',
 'cA_entropy_max',
 'cA_entropy_min',
 'cA_mean',
 'cA_mean_max',
 'cA_mean_min',
 'cA_std',
 'cA_std_max',
 'cA_std_min',
 'cD_energy',
 'cD_energy_max',
 'cD_energy_min',
 'cD_entropy',
 'cD_entropy_max',
 'cD_entropy_min',
 'cD_mean',
 'cD_mean_max',
 'cD_mean_min',
 'cD_std',
 'cD_std_max',
 'cD_std_min',
 'coeff_var',
 'd1_max',
 'd1_mean',
 'd2_max',
 'd2_mean',
 'hjworth_activity',
 'hjworth_complexity',
 'hjworth_mobility',
 'kurtosis',
 'skew']